# Réseaux génératifs

Les réseaux neuronaux récurrents (RNN) et leurs variantes à cellules à portes, comme les cellules de mémoire à long court terme (LSTM) et les unités récurrentes à portes (GRU), ont fourni un mécanisme pour la modélisation du langage, c'est-à-dire qu'ils peuvent apprendre l'ordre des mots et fournir des prédictions pour le mot suivant dans une séquence. Cela nous permet d'utiliser les RNN pour des **tâches génératives**, telles que la génération de texte ordinaire, la traduction automatique et même la génération de légendes pour des images.

Dans l'architecture RNN que nous avons abordée dans l'unité précédente, chaque unité RNN produisait le prochain état caché comme sortie. Cependant, nous pouvons également ajouter une autre sortie à chaque unité récurrente, ce qui nous permettrait de produire une **séquence** (de même longueur que la séquence originale). De plus, nous pouvons utiliser des unités RNN qui n'acceptent pas d'entrée à chaque étape, mais qui prennent simplement un vecteur d'état initial, puis produisent une séquence de sorties.

Dans ce notebook, nous allons nous concentrer sur des modèles génératifs simples qui nous aident à générer du texte. Pour simplifier, construisons un **réseau au niveau des caractères**, qui génère du texte lettre par lettre. Pendant l'entraînement, nous devons prendre un corpus de texte et le diviser en séquences de lettres.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Construire un vocabulaire de caractères

Pour créer un réseau génératif au niveau des caractères, il est nécessaire de diviser le texte en caractères individuels plutôt qu'en mots. Cela peut être réalisé en définissant un tokenizer différent :


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Voyons l'exemple de la façon dont nous pouvons encoder le texte de notre ensemble de données :


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Entraîner un RNN génératif

La manière dont nous allons entraîner un RNN à générer du texte est la suivante. À chaque étape, nous prendrons une séquence de caractères de longueur `nchars` et demanderons au réseau de générer le caractère de sortie suivant pour chaque caractère d'entrée :

![Image montrant un exemple de génération RNN du mot 'HELLO'.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

Selon le scénario spécifique, nous pourrions également vouloir inclure certains caractères spéciaux, tels que *fin de séquence* `<eos>`. Dans notre cas, nous souhaitons simplement entraîner le réseau pour une génération de texte infinie. Par conséquent, nous fixerons la taille de chaque séquence à `nchars` tokens. Ainsi, chaque exemple d'entraînement sera composé de `nchars` entrées et de `nchars` sorties (qui correspondent à la séquence d'entrée décalée d'un symbole vers la gauche). Un minibatch sera constitué de plusieurs de ces séquences.

La manière dont nous générerons les minibatches consiste à prendre chaque texte d'actualité de longueur `l` et à en extraire toutes les combinaisons possibles entrée-sortie (il y aura `l-nchars` combinaisons de ce type). Ces combinaisons formeront un minibatch, et la taille des minibatches variera à chaque étape d'entraînement.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Définissons maintenant le réseau générateur. Il peut être basé sur n'importe quelle cellule récurrente que nous avons abordée dans l'unité précédente (simple, LSTM ou GRU). Dans notre exemple, nous utiliserons un LSTM.

Étant donné que le réseau prend des caractères en entrée et que la taille du vocabulaire est assez petite, nous n'avons pas besoin de couche d'embedding ; une entrée encodée en one-hot peut être directement transmise à la cellule LSTM. Cependant, comme nous passons des numéros de caractères en entrée, nous devons les encoder en one-hot avant de les transmettre au LSTM. Cela se fait en appelant la fonction `one_hot` pendant le passage `forward`. L'encodeur de sortie sera une couche linéaire qui convertira l'état caché en une sortie encodée en one-hot.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Pendant l'entraînement, nous voulons pouvoir échantillonner du texte généré. Pour cela, nous allons définir une fonction `generate` qui produira une chaîne de caractères de longueur `size`, en commençant par la chaîne initiale `start`.

Voici comment cela fonctionne. Tout d'abord, nous passons la chaîne de départ complète à travers le réseau, et nous obtenons l'état de sortie `s` ainsi que le prochain caractère prédit `out`. Comme `out` est encodé en one-hot, nous utilisons `argmax` pour obtenir l'indice du caractère `nc` dans le vocabulaire, puis nous utilisons `itos` pour déterminer le caractère réel et l'ajouter à la liste résultante de caractères `chars`. Ce processus de génération d'un caractère est répété `size` fois pour générer le nombre requis de caractères.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Passons à l'entraînement ! La boucle d'entraînement est presque identique à celle de tous nos exemples précédents, mais au lieu d'afficher la précision, nous affichons un texte généré échantillonné tous les 1000 epochs.

Une attention particulière doit être portée à la manière dont nous calculons la perte. Nous devons calculer la perte en utilisant une sortie encodée en one-hot `out` et le texte attendu `text_out`, qui est la liste des indices de caractères. Heureusement, la fonction `cross_entropy` attend en premier argument la sortie non normalisée du réseau, et en second le numéro de classe, ce qui correspond exactement à ce que nous avons. Elle effectue également une moyenne automatique sur la taille du minibatch.

Nous limitons également l'entraînement à `samples_to_train` échantillons, afin de ne pas attendre trop longtemps. Nous vous encourageons à expérimenter et à essayer un entraînement plus long, éventuellement sur plusieurs epochs (dans ce cas, vous devrez créer une autre boucle autour de ce code).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Cet exemple génère déjà un texte de bonne qualité, mais il peut être encore amélioré de plusieurs façons :

* **Meilleure génération de minibatchs**. La manière dont nous avons préparé les données pour l'entraînement consistait à générer un minibatch à partir d'un seul échantillon. Ce n'est pas idéal, car les minibatchs ont tous des tailles différentes, et certains ne peuvent même pas être générés, car le texte est plus petit que `nchars`. De plus, les petits minibatchs n'exploitent pas suffisamment le GPU. Il serait plus judicieux de prendre un grand bloc de texte à partir de tous les échantillons, de générer ensuite toutes les paires entrée-sortie, de les mélanger, puis de créer des minibatchs de taille égale.

* **LSTM multicouche**. Il est pertinent d'essayer 2 ou 3 couches de cellules LSTM. Comme nous l'avons mentionné dans l'unité précédente, chaque couche de LSTM extrait certains motifs du texte, et dans le cas d'un générateur au niveau des caractères, on peut s'attendre à ce que les couches inférieures du LSTM soient responsables de l'extraction des syllabes, tandis que les couches supérieures s'occupent des mots et des combinaisons de mots. Cela peut être simplement mis en œuvre en passant un paramètre pour le nombre de couches au constructeur LSTM.

* Vous pouvez également expérimenter avec des **unités GRU** pour voir lesquelles donnent de meilleurs résultats, ainsi qu'avec **différentes tailles de couches cachées**. Une couche cachée trop grande peut entraîner un surapprentissage (par exemple, le réseau apprendra le texte exact), tandis qu'une taille trop petite pourrait ne pas produire de bons résultats.


## Génération de texte souple et température

Dans la définition précédente de `generate`, nous choisissions toujours le caractère avec la probabilité la plus élevée comme prochain caractère dans le texte généré. Cela avait pour conséquence que le texte "tournait" souvent en boucle entre les mêmes séquences de caractères, comme dans cet exemple :
```
today of the second the company and a second the company ...
```

Cependant, si nous examinons la distribution de probabilité pour le prochain caractère, il se peut que la différence entre quelques-unes des probabilités les plus élevées ne soit pas énorme, par exemple un caractère peut avoir une probabilité de 0,2, un autre de 0,19, etc. Par exemple, lorsqu'on cherche le prochain caractère dans la séquence '*play*', le caractère suivant pourrait tout aussi bien être un espace ou **e** (comme dans le mot *player*).

Cela nous amène à la conclusion qu'il n'est pas toujours "juste" de sélectionner le caractère avec la probabilité la plus élevée, car choisir le deuxième plus probable pourrait également conduire à un texte cohérent. Il est plus judicieux de **prélever un échantillon** des caractères à partir de la distribution de probabilité donnée par la sortie du réseau.

Ce prélèvement peut être effectué à l'aide de la fonction `multinomial`, qui met en œuvre ce qu'on appelle la **distribution multinomiale**. Une fonction qui implémente cette génération de texte **souple** est définie ci-dessous :


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Nous avons introduit un paramètre supplémentaire appelé **température**, qui est utilisé pour indiquer à quel point nous devons nous en tenir à la probabilité la plus élevée. Si la température est de 1,0, nous effectuons un échantillonnage multinomial équitable, et lorsque la température tend vers l'infini - toutes les probabilités deviennent égales, et nous sélectionnons aléatoirement le prochain caractère. Dans l'exemple ci-dessous, nous pouvons observer que le texte devient dénué de sens lorsque nous augmentons trop la température, et qu'il ressemble à un texte "cyclé" généré de manière rigide lorsqu'il se rapproche de 0.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
